In [8]:
import warnings
warnings.filterwarnings('ignore')

In [9]:
from flowermd.base import Pack, Simulation
from flowermd.library import PPS, BeadSpring
from flowermd.utils import get_target_box_mass_density
import unyt as u

In [10]:
pps_mol = PPS(num_mols=20, lengths=60)

In [11]:
pps_mol.coarse_grain(beads={"A": "c1ccc(S)cc1"})

In [12]:
ff = BeadSpring(
    r_cut=2.5,
    beads={
        "A": dict(epsilon=1, sigma=0.2),
    },
    bonds={
        "A-A": dict(r0=0.64, k=500),
    },
    angles={"A-A-A": dict(t0=2.8, k=50)},
)

In [13]:
ff.hoomd_forces

In [15]:
cg_system = Pack(molecules=pps_mol, density=1, edge=.2, overlap=.2,packing_expand_factor=5)

In [16]:
gsd_path='temp6.gsd'
cg_sim = Simulation(
    initial_state=cg_system.hoomd_snapshot,
    forcefield=ff.hoomd_forces,
    gsd_write_freq=int(2e5/10),
    gsd_file_name=gsd_path,
    reference_values={"length": 1*u.nm, "mass": 1*u.Unit("amu"), "energy":1*u.Unit("kJ/mol")}
)

Initializing simulation state from a gsd.hoomd.Frame.


In [17]:
target_box = get_target_box_mass_density(density=0.5*u.Unit("g/cm**3"), mass=cg_system.mass.to("g"))
print(target_box)

[7.55471261e-07 7.55471261e-07 7.55471261e-07] cm


In [ ]:
cg_sim.run_update_volume(n_steps=3e5, kT=6.0, tau_kt=1.0, final_box_lengths=target_box, period=100)

In [ ]:
cg_sim.flush_writers()

In [ ]:
cg_sim.run_NVT(n_steps=2e5, kT=3.0, tau_kt=1.0)

In [ ]:
cg_sim.box_lengths

In [ ]:
target_box